In [1]:
import matplotlib.pyplot as plt
import torch
import torchvision


from torch import nn
from torchvision import transforms

import engine
import helper_functions
import utils
import data_setup

torch.__version__

torch.Size([4, 10])


c:\Users\danie\GitHub\Retinal_Diseases_Classifier\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'2.6.0+cu118'

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# Setup directory paths to train and test images
train_dir = "data/Train/DISEASE"
val_dir = "data/Val/DISEASE"
test_dir = "data/Test/DISEASE"

In [4]:
effnetb2, effnetb2_transforms = engine.create_pretrained_effnet_model(version="b2", num_classes=4,seed=42)

In [5]:
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                 test_dir=test_dir,
                                                                                                 transforms=effnetb2_transforms,
                                                                                                 batch_size=32)
class_names


['ARMD', 'DR', 'MH', 'ODC']

In [6]:
print(f"Total batches: {len(train_dataloader_effnetb2)}")
print(f"Dataset size: {len(train_dataloader_effnetb2.dataset)}")


Total batches: 31
Dataset size: 961


# Train EffnetB2 Model

In [7]:
# Setup optimizer
optimizer = torch.optim.AdamW(params=effnetb2.parameters(),lr=0.003, weight_decay=1e-5)

# Setup loss function
loss_fn = torch.nn.CrossEntropyLoss()


effnetb2.to(device)
# Set seeds for reproducibility and train the model
helper_functions.set_seeds()
effnetb2_results = engine.train(model=effnetb2,
                                train_dataloader=train_dataloader_effnetb2,
                                test_dataloader=test_dataloader_effnetb2,
                                epochs=5,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                device=device,
                                binary=False)

 20%|██        | 1/5 [00:42<02:49, 42.31s/it]

Epoch: 1 | train_loss : 1.0270 | train_acc : 0.6290 | train_prec : 0.0000 | train_rec : 0.0000 | train_f1 : 0.0000 | test_loss : 1.1040 | test_acc : 0.5920 | test_prec : 0.0000 | test_rec : 0.0000 | test_f1 : 0.0000 | 


 40%|████      | 2/5 [01:34<02:24, 48.20s/it]

Epoch: 2 | train_loss : 1.0180 | train_acc : 0.6835 | train_prec : 0.0000 | train_rec : 0.0000 | train_f1 : 0.0000 | test_loss : 0.7732 | test_acc : 0.7139 | test_prec : 0.0000 | test_rec : 0.0000 | test_f1 : 0.0000 | 


 60%|██████    | 3/5 [02:21<01:35, 47.56s/it]

Epoch: 3 | train_loss : 0.6387 | train_acc : 0.7863 | train_prec : 0.0000 | train_rec : 0.0000 | train_f1 : 0.0000 | test_loss : 0.5501 | test_acc : 0.7979 | test_prec : 0.0000 | test_rec : 0.0000 | test_f1 : 0.0000 | 


 80%|████████  | 4/5 [03:11<00:48, 48.37s/it]

Epoch: 4 | train_loss : 0.4561 | train_acc : 0.8387 | train_prec : 0.0000 | train_rec : 0.0000 | train_f1 : 0.0000 | test_loss : 0.9937 | test_acc : 0.6684 | test_prec : 0.0000 | test_rec : 0.0000 | test_f1 : 0.0000 | 


100%|██████████| 5/5 [03:56<00:00, 47.34s/it]

Epoch: 5 | train_loss : 0.4785 | train_acc : 0.8347 | train_prec : 0.0000 | train_rec : 0.0000 | train_f1 : 0.0000 | test_loss : 0.4988 | test_acc : 0.8174 | test_prec : 0.0000 | test_rec : 0.0000 | test_f1 : 0.0000 | 


# Save model to path

In [8]:
# Save the model
utils.save_model(model=effnetb2,
                 target_dir="models",
                 model_name="pretrained_effnetb2_type_classification.pth")

[INFO] Saving model to: models\pretrained_effnetb2_type_classification.pth


# Check size of the model

In [9]:
from pathlib import Path

# Get the model size in bytes then convert to megabytes
pretrained_effnetb2_model_size = Path("models/pretrained_effnetb2_type_classification.pth").stat().st_size // (1024*1024) # division converts bytes to megabytes (roughly) 
print(f"Pretrained EffNetB2 feature extractor model size: {pretrained_effnetb2_model_size} MB")

Pretrained EffNetB2 feature extractor model size: 29 MB


In [10]:
# Count number of parameters in EffNetB2
effnetb2_total_params = sum(torch.numel(param) for param in effnetb2.parameters())

# Create a dictionary with EffNetB2 statistics
effnetb2_stats = {"test_loss": effnetb2_results["test_loss"][-1],
                  "test_acc": effnetb2_results["test_acc"][-1],
                  "number_of_parameters": effnetb2_total_params,
                  "model_size (MB)": pretrained_effnetb2_model_size}
effnetb2_stats

{'test_loss': 0.4987876519560814,
 'test_acc': 0.8173611111111111,
 'number_of_parameters': 7706630,
 'model_size (MB)': 29}

# Making Predictions

In [11]:
print(f"[INFO] Finding all filepaths ending with '.jpg' in directory: {test_dir}")
test_data_paths = list(Path(test_dir).glob("*/*.png"))
test_data_paths[:5]

[INFO] Finding all filepaths ending with '.jpg' in directory: data/Test/DISEASE


[WindowsPath('data/Test/DISEASE/ARMD/DatasetA_110.png'),
 WindowsPath('data/Test/DISEASE/ARMD/DatasetA_12.png'),
 WindowsPath('data/Test/DISEASE/ARMD/DatasetA_122.png'),
 WindowsPath('data/Test/DISEASE/ARMD/DatasetA_152.png'),
 WindowsPath('data/Test/DISEASE/ARMD/DatasetA_188.png')]

# Create a function to make predictions and store results

# Make predictions with effnetB2

In [12]:
# Make predictions across test dataset with EffNetB2
import predict
effnetb2_test_pred_dicts = predict.pred_and_store(paths=test_data_paths,
                                          model=effnetb2,
                                          transform=effnetb2_transforms,
                                          class_names=class_names,
                                          device="cpu") # make predictions on CPU 

100%|██████████| 306/306 [00:58<00:00,  5.25it/s]


In [13]:
effnetb2_test_pred_dicts[:2]

[{'image_path': WindowsPath('data/Test/DISEASE/ARMD/DatasetA_110.png'),
  'class_name': 'ARMD',
  'pred_prob': 0.8835,
  'pred_class': 'ARMD',
  'time_for_pred': 1.2328,
  'correct': True},
 {'image_path': WindowsPath('data/Test/DISEASE/ARMD/DatasetA_12.png'),
  'class_name': 'ARMD',
  'pred_prob': 0.5127,
  'pred_class': 'DR',
  'time_for_pred': 0.2829,
  'correct': False}]

# Create Pandas Dataframe to inspect

In [18]:
# Turn the test_pred_dicts into a DataFrame
import pandas as pd
effnetb2_test_pred_df = pd.DataFrame(effnetb2_test_pred_dicts)
effnetb2_test_pred_df.head()

,image_path,class_name,pred_prob,pred_class,time_for_pred,correct
0,data\Test\DISEASE\ARMD\DatasetA_110.png,ARMD,0.8835,ARMD,1.2328,True
1,data\Test\DISEASE\ARMD\DatasetA_12.png,ARMD,0.5127,DR,0.2829,False
2,data\Test\DISEASE\ARMD\DatasetA_122.png,ARMD,0.5137,ARMD,0.2444,True
3,data\Test\DISEASE\ARMD\DatasetA_152.png,ARMD,0.9939,DR,0.1636,False
4,data\Test\DISEASE\ARMD\DatasetA_188.png,ARMD,0.5720,ARMD,0.1823,True


In [19]:
effnetb2_test_pred_df.correct.value_counts()

correct
True     252
False     54
Name: count, dtype: int64

In [20]:
effnetb2_average_time_per_pred = round(effnetb2_test_pred_df.time_for_pred.mean(), 4)
print(f"EffNetB2 average time per prediction: {effnetb2_average_time_per_pred} seconds")

EffNetB2 average time per prediction: 0.1896 seconds


# Using Gradio Demo

In [50]:
import gradio as gr
print(f"Gradio version: {gr.__version__}")

Gradio version: 5.21.0


inputs -> ML model -> outputs  
input: image -> transform -> predict with EffNetB2 -> output: pred, pred prob, time taken

In [51]:
# Put EffNetB2 on CPU
effnetb2.to("cpu") 

# Check the device
next(iter(effnetb2.parameters())).device

device(type='cpu')

# Create Predict Function

In [57]:
from typing import Tuple, Dict

def predict(img) -> Tuple[Dict, float]:
    """Transforms and performs a prediction on img and returns prediction and time taken.
    """
    start_time = timer()
    
    # transform image and add batch dim
    img = effnetb2_transforms(img).unsqueeze(0)
    
    # put model in eval mode and turn on inference mode
    effnetb2.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(effnetb2(img), dim = 1)
        
    # create prediction label and prediction prob
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    pred_time = round(timer() - start_time, 5)
    
    return pred_labels_and_probs, pred_time
    

In [58]:
import random

# Randomly select a test image path
random_image_path = random.sample(test_data_paths, k=1)[0]

# open image
image = Image.open(random_image_path)
print(f"[INFO] Predicting on image at path: {random_image_path}\n")

# Predict on the target image and print out the outputs
pred_dict, pred_time = predict(img=image)
print(f"Prediction label and probability dictionary: \n{pred_dict}")
print(f"Prediction time: {pred_time} seconds")

[INFO] Predicting on image at path: data\pizza_steak_sushi_20_percent\test\steak\289822.jpg

Prediction label and probability dictionary: 
{'pizza': 0.004519496578723192, 'steak': 0.9843215942382812, 'sushi': 0.011158893816173077}
Prediction time: 0.06489 seconds


In [59]:
# Create a list of example inputs to our Gradio demo
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

[['data\\pizza_steak_sushi_20_percent\\test\\sushi\\2352914.jpg'],
 ['data\\pizza_steak_sushi_20_percent\\test\\sushi\\472912.jpg'],
 ['data\\pizza_steak_sushi_20_percent\\test\\pizza\\724290.jpg']]

# Building a Gradio Interface

In [60]:
# Create title, description and article strings
title = "FoodVision Mini 🍕🥩🍣"
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as pizza, steak or sushi."
article = "Created at [09. PyTorch Model Deployment](https://www.learnpytorch.io/09_pytorch_model_deployment/)."

In [61]:
# Create the Gradio demo
demo = gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type="pil"), # what are the inputs?
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    examples=example_list, 
                    title=title,
                    description=description,
                    article=article)

In [63]:
# Launch the demo!
#demo.launch(debug=False, # print errors locally?
#           share=True) # generate a publically shareable URL?

# Create Demo Directory to deploy on Hugging Face Spaces

In [65]:
import shutil
from pathlib import Path

# Create FoodVision mini demo path
foodvision_mini_demo_path = Path("demos/foodvision_mini/")

# Remove files that might already exist there and create new directory
if foodvision_mini_demo_path.exists():
    shutil.rmtree(foodvision_mini_demo_path)
# If the file doesn't exist, create it anyway
foodvision_mini_demo_path.mkdir(parents=True, 
                                exist_ok=True)

In [66]:
# 1. Create an examples directory
foodvision_mini_examples_path = foodvision_mini_demo_path / "examples"
foodvision_mini_examples_path.mkdir(parents=True, exist_ok=True)

# 2. Collect three random test dataset image paths
foodvision_mini_examples = [Path('data/pizza_steak_sushi_20_percent/test/sushi/592799.jpg'),
                            Path('data/pizza_steak_sushi_20_percent/test/steak/3622237.jpg'),
                            Path('data/pizza_steak_sushi_20_percent/test/pizza/2582289.jpg')]

# 3. Copy the three random images to the examples directory
for example in foodvision_mini_examples:
    destination = foodvision_mini_examples_path / example.name
    print(f"[INFO] Copying {example} to {destination}")
    shutil.copy2(src=example, dst=destination)

[INFO] Copying data\pizza_steak_sushi_20_percent\test\sushi\592799.jpg to demos\foodvision_mini\examples\592799.jpg
[INFO] Copying data\pizza_steak_sushi_20_percent\test\steak\3622237.jpg to demos\foodvision_mini\examples\3622237.jpg
[INFO] Copying data\pizza_steak_sushi_20_percent\test\pizza\2582289.jpg to demos\foodvision_mini\examples\2582289.jpg


In [67]:
import os

# Get example filepaths in a list of lists
example_list = [["examples/" + example] for example in os.listdir(foodvision_mini_examples_path)]
example_list

[['examples/2582289.jpg'], ['examples/3622237.jpg'], ['examples/592799.jpg']]

In [69]:
# Create a source path for our target model
effnetb2_foodvision_mini_model_path = "models/09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth"

# Create a destination path for our target model 
effnetb2_foodvision_mini_model_destination = foodvision_mini_demo_path / effnetb2_foodvision_mini_model_path.split("/")[1]

# Try to move the file
try:
    print(f"[INFO] Attempting to move {effnetb2_foodvision_mini_model_path} to {effnetb2_foodvision_mini_model_destination}")
    
    # Move the model
    shutil.move(src=effnetb2_foodvision_mini_model_path, 
                dst=effnetb2_foodvision_mini_model_destination)
    
    print(f"[INFO] Model move complete.")

# If the model has already been moved, check if it exists
except:
    print(f"[INFO] No model found at {effnetb2_foodvision_mini_model_path}, perhaps its already been moved?")
    print(f"[INFO] Model exists at {effnetb2_foodvision_mini_model_destination}: {effnetb2_foodvision_mini_model_destination.exists()}")

[INFO] Attempting to move models/09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth to demos\foodvision_mini\09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth
[INFO] No model found at models/09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth, perhaps its already been moved?
[INFO] Model exists at demos\foodvision_mini\09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth: True


'Compress-Archive' is not recognized as an internal or external command,
operable program or batch file.
